# CORDEX Regional Climate Projections

...

In [3]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Create DHIS2 client connection
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district"
)
client = DHIS2Client(settings=cfg)

# Verify connection
info = client.get_system_info()
print("Current DHIS2 version:", info["version"])

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [2]:
import dhis2eo
import dhis2eo.org_units
import dhis2eo.data.cds
import dhis2eo.synch
import dhis2eo.utils.aggregate

In [3]:
org_unit_level = 2
#org_units_geojson = client.get_org_units_geojson(level=org_unit_level)
#org_units = dhis2eo.org_units.from_dhis2_geojson(org_units_geojson)
org_units = dhis2eo.org_units.from_file('../data/sierra-leone-districts.geojson', org_unit_id_col=None, name_col='name', level=org_unit_level)
org_units

,org_unit_id,name,level,geometry
0,A637R4mf9Y7,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,Y8ZKS1GUD8y,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,PMpNYzoFnNb,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,WJbZWVVFMk4,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,QPq0LgGszk5,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,Lk7oE0kOkgq,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,XiBg02FJkXY,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,DomWoL7s7fI,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,ViQOmIhjxxC,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,XSBBAlXCyFG,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


In [ ]:
import earthkit.data
import json

def get_daily_cordex_data(region, scenario, resolution, gcm_model, rcm_model, start_year, end_year, org_units, variables=None):
    # get default variables
    variables = variables or [
        "2m_air_temperature",
        "2m_relative_humidity",
        "mean_precipitation_flux"
    ]
        
    # construct the query parameters
    params = {
        "domain": region,
        "experiment": scenario,
        "horizontal_resolution": resolution,
        "temporal_resolution": "daily_mean",
        "variable": variables,
        "gcm_model": gcm_model,
        "rcm_model": rcm_model,
        "ensemble_member": "r1i1p1",
        "start_year": [str(start_year)],
        "end_year": [str(end_year)],
    }

    # download the data
    print(f'Downloading climate projection data from CDS API...')
    print(f'Request parameters: \n{json.dumps(params)}')
    data = earthkit.data.from_source("cds",
        "projections-cordex-domains-single-levels",
        **params
    )
    #data.save()

    # crop the data to our org units bbox
    print(f'Clipping data to bbox...')
    bbox = list(org_units.total_bounds)
    minx,miny,maxx,maxy = bbox
    clipped = data.rio.clip_bbox(minx=minx, miny=miny, maxx=maxx, maxy=maxy)
    
    # return
    return clipped

region = 'africa'
scenario = 'rcp_2_6'
resolution = '0_22_degree_x_0_22_degree'
gcm_model = 'mohc_hadgem2_es'
rcm_model = 'clmcom_kit_cclm5_0_15'
start_year = 2026
end_year = 2030
data = get_daily_cordex_data(region, scenario, resolution, gcm_model, rcm_model, start_year, end_year, org_units)
print(data)
data.to_xarray()

Request parameters: 
{"domain": "africa", "experiment": "rcp_2_6", "horizontal_resolution": "0_22_degree_x_0_22_degree", "temporal_resolution": "daily_mean", "variable": ["2m_air_temperature", "2m_relative_humidity", "mean_precipitation_flux"], "gcm_model": "mohc_hadgem2_es", "rcm_model": "clmcom_kit_cclm5_0_15", "ensemble_member": "r1i1p1", "start_year": ["2026"], "end_year": ["2030"]}


c:\Users\karimba\AppData\Local\miniconda3\envs\climate-tools-new\Lib\site-packages\ecmwf\datastores\client.py:87: UserWarning: HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1032)')))
  warnings.warn(str(exc), UserWarning)
2025-10-21 18:18:03,253 INFO Request ID is aa557a9d-369d-4200-90f1-b1b5a6c85e8a
2025-10-21 18:18:03,380 INFO status has been updated to accepted
2025-10-21 18:18:12,075 INFO status has been updated to running


SSLError: HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/retrieve/v1/jobs/aa557a9d-369d-4200-90f1-b1b5a6c85e8a?log=True&request=True (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1032)')))